# 03. Merge & Normalize EUNIS Data

**Goal:** Create a single, clean GeoDataFrame from the 20+ raw files.

**Tasks:**
1. Investigate `Code_tif` vs `Code_` (are they duplicates?).
2. Define a **Column Mapping Strategy** to unify disparate names (e.g. `HabitatCod` -> `EUNIS_CODE`).
3. Apply the mapping and merge all datasets.
4. Save the result as a single Parquet or GeoPackage file (more efficient than GeoJSON).

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Setup paths
raw_dir = Path("../data/raw/eunis")
processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

files = sorted(list(raw_dir.glob("*.geojson")))
print(f"Processing {len(files)} files.")

Processing 8 files.


## 1. Investigation: What is `Code_tif`?
Let's load a dataset that contains both `Code_` and `Code_tif` (e.g., 'EUNIS_Forest_Distribution_point' usually has strictly structured attributes) to see if they carry the same information.

In [2]:
# Load a sample file (ensure it's one that has Code_tif based on previous analysis)
# We iterate until we find one with both columns
sample_gdf = None
target_file = None

for p in files:
    gdf_test = gpd.read_file(p, rows=5)
    if 'Code_' in gdf_test.columns and 'Code_tif' in gdf_test.columns:
        target_file = p
        sample_gdf = gpd.read_file(p, rows=100) # Load 100 rows for check
        break

if sample_gdf is not None:
    print(f"Checking correlation in: {target_file.name}")
    print(sample_gdf[['Code_', 'Code_tif', 'Name_']].head(10))
    
    # Check if they match 1:1
    # Usually Code_tif is an Integer ID, while Code_ is a String (e.g., T1.1)
    print("\nUnique types:")
    print(sample_gdf[['Code_', 'Code_tif']].dtypes)
else:
    print("No dataset found with both columns for comparison.")

Checking correlation in: EUNIS_Forest_Distribution_point.geojson
  Code_      Code_tif                                        Name_
0   T11  T11_data.tif  Temperate Salix and Populus riparian forest
1   T11  T11_data.tif  Temperate Salix and Populus riparian forest
2   T11  T11_data.tif  Temperate Salix and Populus riparian forest
3   T11  T11_data.tif  Temperate Salix and Populus riparian forest
4   T11  T11_data.tif  Temperate Salix and Populus riparian forest
5   T11  T11_data.tif  Temperate Salix and Populus riparian forest
6   T11  T11_data.tif  Temperate Salix and Populus riparian forest
7   T11  T11_data.tif  Temperate Salix and Populus riparian forest
8   T11  T11_data.tif  Temperate Salix and Populus riparian forest
9   T11  T11_data.tif  Temperate Salix and Populus riparian forest

Unique types:
Code_       str
Code_tif    str
dtype: object


**Decision on `Code_tif`:** # Usually, `Code_tif` is a raster grid ID (integer) used for spatial analysis, while `Code_` is the semantic label. 

*Strategy:* We will **keep both** but rename `Code_tif` to `RASTER_ID` to avoid confusion. The main analysis will use `EUNIS_CODE`.

## 2. Define Normalization Logic
We create a mapping dictionary. Any column not in this map (and not in `keep_columns`) will be dropped.

In [3]:
# 1. Target Schema (Renaming)
# Format: 'Original_Name': 'New_Unified_Name'
column_mapping = {
    # The Codes
    'Code_': 'EUNIS_CODE',
    'Code_2018': 'EUNIS_CODE',
    'HabitatCod': 'EUNIS_HABITAT_CODE',  # Merging this dominant field here
    
    # The Names
    'Name': 'EUNIS_NAME',
    'Name_': 'EUNIS_NAME',
    'Name_2018': 'EUNIS_NAME',
    'Descriptio': 'EUNIS_DESC',
    'Description': 'EUNIS_DESC',
    
    # Technical / Other
    'Code_tif': 'RASTER_ID',
    'Level': 'LEVEL',
    'YEAR': 'YEAR',
    'PRECISION': 'PRECISION_M', # Explicit unit
    'PLOTOBSID': 'PLOT_ID'
}

# 2. Columns to always keep if they exist (don't rename, just keep)
keep_columns = ['geometry', 'OBJECTID']

def normalize_gdf(gdf, filename):
    """Renames columns and selects only the relevant subset."""
    
    # 1. Rename columns based on mapping
    # We use a loop because a file might have 'Code_' OR 'HabitatCod', 
    # and simple .rename() works for both mapping to the same target.
    gdf = gdf.rename(columns=column_mapping)
    
    # 2. Add Source Metadata (Crucial for debugging later!)
    gdf['SOURCE_FILE'] = filename
    
    # 3. Filter Columns
    # Identify which of our target columns actually exist in this dataframe
    target_cols = set(column_mapping.values())
    existing_cols = set(gdf.columns)
    
    # Valid columns = (Renamed Targets) OR (Keep Columns)
    cols_to_select = [c for c in existing_cols if c in target_cols or c in keep_columns or c == 'SOURCE_FILE']
    
    return gdf[cols_to_select]

## 3. Execute Merge
We stream-load, normalize, and collect into a list (or merge iteratively if RAM is tight, but 20 files should fit in memory if they aren't massive).

In [4]:
normalized_dfs = []

print("Starting normalization...")
for p in files:
    try:
        # Load
        gdf_raw = gpd.read_file(p)
        
        # Normalize
        gdf_clean = normalize_gdf(gdf_raw, p.name)
        
        # Fix CRS if needed (usually EPSG:3035 for EEA data, but verify)
        if normalized_dfs and gdf_clean.crs != normalized_dfs[0].crs:
            print(f"⚠️ CRS Mismatch in {p.name}. Reprojecting...")
            gdf_clean = gdf_clean.to_crs(normalized_dfs[0].crs)
            
        normalized_dfs.append(gdf_clean)
        print(f"✅ Processed {p.name} | Rows: {len(gdf_clean)}")
        
    except Exception as e:
        print(f"❌ Failed {p.name}: {e}")

# Concatenate all
print("\nMerging datasets...")
full_gdf = pd.concat(normalized_dfs, ignore_index=True)

print(f"Final Dataset Shape: {full_gdf.shape}")
print(f"CRS: {full_gdf.crs}")

Starting normalization...
✅ Processed EUNIS_Coastal_Distribution_point.geojson | Rows: 34102
✅ Processed EUNIS_Forest_Distribution_point.geojson | Rows: 220601
✅ Processed EUNIS_Grassland_Distribution_point.geojson | Rows: 298801
✅ Processed EUNIS_Hethland_Distribution_point.geojson | Rows: 42918
✅ Processed EUNIS_Saltmarshes_Distribution_point.geojson | Rows: 21141
✅ Processed EUNIS_Sparsely_Distribution_point.geojson | Rows: 6521
✅ Processed EUNIS_Vegetated_Distribution_point.geojson | Rows: 79069
✅ Processed EUNIS_Wetlands_Distribution_point.geojson | Rows: 92468

Merging datasets...
Final Dataset Shape: (795621, 12)
CRS: EPSG:4326


## 4. Post-Merge Validation
Let's check if we still have NaNs in the critical `EUNIS_CODE` column.

In [5]:
# Check for nulls in key fields
print(full_gdf[['EUNIS_CODE', 'EUNIS_NAME']].info())

missing_codes = full_gdf[full_gdf['EUNIS_CODE'].isnull()]
if not missing_codes.empty:
    print(f"\n⚠️ Warning: {len(missing_codes)} rows have no EUNIS_CODE!")
    print(missing_codes['SOURCE_FILE'].value_counts())
else:
    print("\n✅ Success: All rows have a EUNIS_CODE.")

# Verify unique levels if the column exists
if 'LEVEL' in full_gdf.columns:
    print("\nHabitat Levels:", full_gdf['LEVEL'].unique())

<class 'pandas.DataFrame'>
RangeIndex: 795621 entries, 0 to 795620
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   EUNIS_CODE  795621 non-null  str  
 1   EUNIS_NAME  789996 non-null  str  
dtypes: str(2)
memory usage: 12.1 MB
None

✅ Success: All rows have a EUNIS_CODE.

Habitat Levels: [nan  3.]


## 5. Save Results
We save to **Parquet** (fast, efficient for large data) or **GeoPackage**. 
Avoid GeoJSON for the merged file (it will be huge and slow).

In [7]:
output_path = processed_dir / "eunis_merged_all.parquet"

# Save as GeoParquet
full_gdf.to_parquet(output_path)
print(f"Saved merged dataset to {output_path}")

# Optional: Save a small sample as GeoJSON for quick visualization in QGIS/GitHub
full_gdf.sample(1000).to_file(processed_dir / "eunis_sample_1k.geojson", driver="GeoJSON")

Saved merged dataset to ../data/processed/eunis_merged_all.parquet
